In [7]:
import pandas as pd
import tidydata
import warnings

In [6]:
#  выгрузим список индикаторв с сайта федстата
indicators = tidydata.indicators_scrapper(first_ind = '58536', last_ind = '58468')

In [ ]:
#  выгрузим файлы данных с сайта федстата
tidydata.fedstat_files_loader(indicators)

In [8]:
#  преобразуем данные
warnings.filterwarnings('ignore')
data_dir = '/home/dad/projects/tgu/test_task/data/'
c1 = tidydata.combiner_1(data_dir)
c2 = tidydata.combiner_8(data_dir)
c3 = tidydata.combiner_9(data_dir)
c4 = tidydata.combiner_12(data_dir)
c5 = tidydata.combiner_51(data_dir)

In [3]:
result = pd.concat([c1, c2, c3, c4, c5], axis=0)     #  соединим промежуточные результаты в одну таблицу
result['Значение'] = result['Значение'].apply( lambda x: ','.join(str(x).split('.')) ) # замена разделитля десятичной части дроби на запятую
result.dropna(subset=['Значение'], inplace=True) #  отфильтруем пропущенные значения
filter = result['Значение'].isin([0, 'Null'])    #  отфильтруем нулевые значения
result[~filter].to_csv('tidydata.csv', encoding='utf-8', sep=';', decimal=',', index = False) #  сохраним результаты в файл csv